In [40]:
from dotenv import load_dotenv
load_dotenv()
cohere_api_key=os.getenv("COHERE_API_KEY")
alpha_vantage_api_key=os.getenv("ALPHA_VANTAGE_API_KEY")

In [59]:
import requests

# Function to fetch financial data from Alpha Vantage API
def fetch_financial_data(api_key, ticker):
    """
    Fetch financial data for the given ticker using Alpha Vantage API.
    Returns a dictionary with revenue, net income, liabilities, and assets.
    """
    base_url = "https://www.alphavantage.co/query"
    income_params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": api_key
    }
    balance_params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": api_key
    }

    # Fetch Income Statement
    income_response = requests.get(base_url, params=income_params)
    if income_response.status_code != 200:
        return {"error": "Failed to fetch income statement data"}
    income_data = income_response.json()

    # Fetch Balance Sheet
    balance_response = requests.get(base_url, params=balance_params)
    if balance_response.status_code != 200:
        return {"error": "Failed to fetch balance sheet data"}
    balance_data = balance_response.json()

    # Extract relevant data
    try:
        latest_income = income_data["annualReports"][0]
        latest_balance = balance_data["annualReports"][0]

        return {
            "revenue": float(latest_income["totalRevenue"]),
            "net_income": float(latest_income["netIncome"]),
            "liabilities": float(latest_balance["totalLiabilities"]),
            "assets": float(latest_balance["totalAssets"])
        }
    except (KeyError, IndexError):
        return {"error": "Incomplete financial data"}

# Function to compute financial ratios
def compute_ratios(data):
    """
    Compute financial ratios from the provided data.
    """
    revenue = data.get("revenue", 1)  # Avoid division by zero
    net_income = data.get("net_income", 0)
    liabilities = data.get("liabilities", 0)
    assets = data.get("assets", 1)

    return {
        "Return on Equity (ROE)": net_income / (assets - liabilities) if assets != liabilities else 0,
        "Return on Assets (ROA)": net_income / assets,
        "Gross Margin": (revenue - liabilities) / revenue if revenue else 0,
        "Net Profit Margin": net_income / revenue if revenue else 0
    }

# Define tools for the agent
def fetch_financial_tool(input_text):
    """
    Fetch and calculate financial ratios based on stock ticker input.
    """
    ticker = input_text.upper()
    # api_key = "YOUR_ALPHA_VANTAGE_API_KEY"  # Replace with your Alpha Vantage API key
    data = fetch_financial_data(alpha_vantage_api_key, ticker)
    if "error" in data:
        return data["error"]
    ratios = compute_ratios(data)
    return ratios

In [60]:
fetch_financial_tool(ticker)

{'Return on Equity (ROE)': 1.6459350307287095,
 'Return on Assets (ROA)': 0.25682503150857583,
 'Gross Margin': 0.21227000140652372,
 'Net Profit Margin': 0.23971255769943867}

In [29]:
main()

Financial Analysis for AAPL:
Ticker: AAPL
Return on Equity (ROE): 164.59%
Return on Assets (ROA): 25.68%
Gross Margin: 46.21%
Net Profit Margin: 23.97%
Debt to Equity Ratio: 5.41


In [78]:
import cohere
import requests
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.prompts import PromptTemplate
from langchain.llms import Cohere
import yfinance as yf

# Initialize Cohere API (for evaluation or financial analysis)
cohere_client = Cohere(model="command")

# Function to fetch financial data (using the Alpha Vantage API as an example)
def get_financial_data2(symbol: str):
    api_key = alpha_vantage_api_key
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    return data

def get_financial_data(symbol: str):
    # Fetch data using yfinance
    stock_data = yf.Ticker(symbol).history(period="5d")  # Last 5 days of stock data
    return stock_data.to_json()

# Define a function to evaluate the financial data (using Cohere)
def evaluate_data(data):
    # Use Cohere to evaluate the stock data (could be more specific analysis in real-world cases)
    prompt = f"Please evaluate the following stock data:\n\n{data}\n\nProvide insights or a summary."
    response = cohere_client.generate(
        model='xlarge', prompt=prompt, max_tokens=150
    )
    return response.generations[0].text

# Create the tools
tools = [
    Tool(
        name="Fetch Financial Data",
        func=get_financial_data,
        description="Fetch financial data for a specific stock symbol."
    ),
    Tool(
        name="Evaluate Data",
        func=evaluate_data,
        description="Evaluate the fetched financial data using Cohere's model."
    ),
]

# Initialize the Cohere model as an LLM in LangChain
cohere_llm = Cohere(model="command")

# Initialize the LangChain agent (using Cohere as the LLM)
agent = initialize_agent(tools, cohere_llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Function to run the agent
def run_agent(stock_symbol: str):
    result = agent.run(stock_symbol)
    return result

# Example usage
if __name__ == "__main__":
    stock_symbol = "LUV"  # Example stock symbol for Apple
    result = run_agent(stock_symbol)
    print(result)




> Entering new AgentExecutor chain...
 This is the stock symbol for Southwest Airlines, and this is in the transportation industry which has been suffering due to the war in Ukraine and the rise of Covid-19 infections in Asia again. 
Action: Fetch Financial Data
Action Input: LUV
Observation: {"Open":{"1731560400000":32.5499992371,"1731646800000":32.25,"1731906000000":32.5999984741,"1731992400000":31.9200000763,"1732078800000":32.2400016785},"High":{"1731560400000":33.0800018311,"1731646800000":32.6599998474,"1731906000000":32.7799987793,"1731992400000":32.5600013733,"1732078800000":32.3199996948},"Low":{"1731560400000":32.2900009155,"1731646800000":31.7600002289,"1731906000000":32.0299987793,"1731992400000":31.75,"1732078800000":31.4500007629},"Close":{"1731560400000":32.4199981689,"1731646800000":32.5999984741,"1731906000000":32.3300018311,"1731992400000":32.3600006104,"1732078800000":31.7700004578},"Volume":{"1731560400000":6263500,"1731646800000":6251400,"1731906000000":6988900,"

TypeError: BaseLLM.generate() missing 1 required positional argument: 'prompts'

In [ ]:
import requests

def fetch_financial_data(api_key, ticker):
    """
    Fetch financial data for the given ticker using Alpha Vantage API.
    Returns a dictionary with expanded financial metrics.
    """
    base_url = "https://www.alphavantage.co/query"
    income_params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": api_key
    }
    balance_params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": api_key
    }
    cash_flow_params = {
        "function": "CASH_FLOW",
        "symbol": ticker,
        "apikey": api_key
    }

    # Fetch Income Statement
    income_response = requests.get(base_url, params=income_params)
    if income_response.status_code != 200:
        return {"error": "Failed to fetch income statement data"}
    income_data = income_response.json()

    # Fetch Balance Sheet
    balance_response = requests.get(base_url, params=balance_params)
    if balance_response.status_code != 200:
        return {"error": "Failed to fetch balance sheet data"}
    balance_data = balance_response.json()

    # Fetch Cash Flow Statement
    cash_flow_response = requests.get(base_url, params=cash_flow_params)
    if cash_flow_response.status_code != 200:
        return {"error": "Failed to fetch cash flow data"}
    cash_flow_data = cash_flow_response.json()

    # Extract relevant data
    try:
        latest_income = income_data["annualReports"][0]
        latest_balance = balance_data["annualReports"][0]
        latest_cash_flow = cash_flow_data["annualReports"][0]
        
        return {
            "revenue": float(latest_income["totalRevenue"]),
            "net_income": float(latest_income["netIncome"]),
            "interest_expense": (latest_income.get("interestExpense", 0)),
            "total_liabilities": float(latest_balance["totalLiabilities"]),
            "total_debt": float(latest_balance.get("totalDebt", 0)),
            "total_equity": float(latest_balance.get("totalShareholderEquity", 0)),
            "current_assets": float(latest_balance.get("totalCurrentAssets", 0)),
            "current_liabilities": float(latest_balance.get("totalCurrentLiabilities", 0)),
            "inventory": float(latest_balance.get("inventory", 0)),
            "cash_and_equivalents": float(latest_balance.get("cashAndCashEquivalentsAtCarryingValue", 0)),
            "total_assets": float(latest_balance["totalAssets"]),
            "operating_cash_flow": float(latest_cash_flow.get("operatingCashflow", 0))
        }
    except (KeyError, IndexError):
        return {"error": "Incomplete financial data"}

def compute_ratios(data):
    """
    Compute enhanced financial ratios from the provided data.
    Handles potential division by zero scenarios.
    """
    # Extract values with safe defaults
    revenue = data.get("revenue", 1)  # Avoid division by zero
    net_income = data.get("net_income", 0)
    total_liabilities = data.get("total_liabilities", 0)
    total_debt = data.get("total_debt", 0)
    total_equity = data.get("total_equity", 1)
    current_assets = data.get("current_assets", 0)
    current_liabilities = data.get("current_liabilities", 1)
    inventory = data.get("inventory", 0)
    cash_and_equivalents = data.get("cash_and_equivalents", 0)
    total_assets = data.get("total_assets", 1)
    interest_expense = data.get("interest_expense", 0)
    operating_cash_flow = data.get("operating_cash_flow", 0)

    # Compute ratios with error handling
    ratios = {
        # Existing Ratios
        "Return on Equity (ROE)": net_income / total_equity if total_equity != 0 else 0,
        "Return on Assets (ROA)": net_income / total_assets if total_assets != 0 else 0,
        "Gross Margin": (revenue - total_liabilities) / revenue if revenue else 0,
        "Net Profit Margin": net_income / revenue if revenue else 0,
        
        # New Financial Ratios
        "Debt to Equity Ratio": total_debt / total_equity if total_equity != 0 else float('inf'),
        
        "Interest Coverage Ratio": (net_income + interest_expense) / interest_expense if interest_expense != 0 else float('inf'),
        
        "Current Ratio": current_assets / current_liabilities if current_liabilities != 0 else float('inf'),
        
        "Quick Ratio": (current_assets - inventory) / current_liabilities if current_liabilities != 0 else float('inf'),
        
        "Inventory Turnover": revenue / inventory if inventory != 0 else float('inf'),
        
        "Asset Turnover": revenue / total_assets if total_assets != 0 else float('inf')
    }
    
    return ratios

def fetch_financial_tool(input_text):
    """
    Fetch and calculate financial ratios based on stock ticker input.
    """
    ticker = input_text.upper()
    # api_key = "YOUR_ALPHA_VANTAGE_API_KEY"  # Replace with your Alpha Vantage API key
    data = fetch_financial_data(alpha_vantage_api_key, ticker)
    if "error" in data:
        return data["error"]
    ratios = compute_ratios(data)
    return ratios

tools = [
    Tool(
        name="Fetch Financial Ratios",
        func=fetch_financial_tool,
        description="Fetches financial data and computes comprehensive ratios based on a stock ticker."
    )
]